#### <font color=#FF6600> **1. Import relative packages** <font>

In [1]:
import numpy as np
import pandas as pd

#### <font color=#FF6600> **2. Read data** <font>

In [54]:
# 資料來源: https://www.kaggle.com/code/tzulingli/rmf-analysis-and-k-means-clustering/data
df = pd.read_csv('./data/scanner_data.csv', dtype={'Customer_ID': 'str'})
df.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


In [55]:
print(f"""
    The shape of the dataset is: {df.shape}.
    There are {df.Customer_ID.nunique():,} unique customers
    and {df.SKU.nunique():,} unique skus in {df.SKU_Category.nunique():,} unique categories.
""")


    The shape of the dataset is: (131706, 8).
    There are 22,625 unique customers
    and 5,242 unique skus in 187 unique categories.



#### <font color=#FF6600> **3. Define Diversity function** <font>

In [60]:
def diversity_cal(data, index_col, cat_col, target_col):
    """
    :param data: input dataframe including index_col, cat_col, target_col
    :param index_col: customer id column
    :param cat_col: category id column
    :param target_col: the column that you want to sum to calculate diversity
    :return df_diversity: diversity data that including index_col & diversity
    """
    def hhi(series):
        return np.square(series/series.sum()).sum() * 10000
    df_sum_by_cat = data.groupby([index_col, cat_col],
                            as_index=False,
                            sort=False)[target_col].agg({'sum_by_cat': 'sum'})
    df_diversity = df_sum_by_cat.groupby([index_col]).agg({'sum_by_cat': hhi}).reset_index()
    df_diversity['diversity'] = 10000 - df_diversity.sum_by_cat
    df_diversity.drop(columns=['sum_by_cat'], inplace=True)
    return df_diversity

#### <font color=#FF6600> **4. Calculate Diversity Index** <font>

In [61]:
df_diversity = diversity_cal(data=df,
                            index_col='Customer_ID',
                            cat_col='SKU_Category',
                            target_col='Sales_Amount')

In [62]:
df_diversity.head()

,Customer_ID,diversity
0,1,4582.006383
1,10,0.000000
2,100,4642.682090
3,1000,0.000000
4,10000,6269.218434
